Install relevant modules and packages 

In [ ]:
!apt-get install -y dcm2niix
!apt-get install -y parallel

!pip install nibabel

import os
import glob
import nibabel as nib

# install freesurfer (might take a few minutes)
!apt-get install -y wget
!apt-get install -y grep

# if you are having issues downloading freesurfer, it is likely due to the version selected here
!wget -O freesurfer.tar.gz https://freesurfer.net/pub/dist/freesurfer/7.5.0/freesurfer-macOS-darwin_arm64-7.5.0.tar.gz
!tar -xzf freesurfer.tar.gz

# set the relevant freesurfer directories
os.environ['FREESURFER_HOME'] = '/content/freesurfer'
os.environ['SUBJECTS_DIR'] = '/content/subjects'
os.environ['FS_LICENSE'] = '/content/license.txt'  # Add your license file path if you have one
os.environ['PATH'] += ':/content/freesurfer/bin'

Get anonymized (T1 weighted) dicom files for analysis, and create temporary folders to store these and other files

In [ ]:
# clone the repository to get the raw dicoms
!git clone https://github.com/datalad/example-dicom-structural

# create folders for the NIfTI conversions and corresponding recons
!mkdir /content/nii_files/
!mkdir /content/freesurfer_output/

Convert the DICOMs to NIfTIs

In [ ]:
INPUT_PATH = '/content/example-dicom-structural/dicoms/'
OUTPUT_PATH = '/content/nii_files/'

# grep ignores warnings regarding the manufacturer (since we're using sample dicoms)
!dcm2niix -o '{OUTPUT_PATH}' '{INPUT_PATH}' | grep -v “Unknown Manufacturer”

Reconstruct the Nii files.

Parallel is used here for demonstration purposes, but since we only have a single dicom-containing folder, it doesn't do anything in practice. 

Using parallel in this way allows you to recon multiple subjects (each with their own folder) simultaneously by recruiting n CPU cores (determined by --jobs n)

In [ ]:
RECON_PATH = '/content/drive/MyDrive/freesurfer_output'

import glob
ALL_NIIS = glob.glob('/content/drive/MyDrive/nii_files/*')

parallel --jobs 1 'recon-all -i {} -sd '"$RECON_PATH"' -all' ::: $ALL_NIIS